In [1]:
%load_ext autoreload
%autoreload 2   

In [ ]:
from typing import Any
import re
from enum import Enum

In [16]:
from tools.writer import EnumEntry, enum_file, write_enum_class, AUTO_ENUMS_PATH

In [4]:
from statscan.wds.client import WDS

In [5]:
client = WDS()

In [6]:
codesets: dict = await client.getCodeSets()

In [7]:
o: dict[str, list[dict[str, str]]] = codesets['object']

In [8]:
def get_keys(entry_dict: dict[str, Any]) -> tuple[str, str, set[str]]:
    '''
    get the key-key, value-key, and remaining keys
    '''
    remaining_keys = set()
    key_key = None
    value_key = None

    for k in entry_dict.keys():
        if k.endswith("DescEn"):
            key_key = k
        elif k.endswith("Code"):
            value_key = k
        else:
            remaining_keys.add(k)
    
    if key_key is None:
        for k in entry_dict.keys():
            if k.endswith("En"):
                key_key = k
                remaining_keys.remove(k)
                break
        if key_key is None:
            raise ValueError(f"Could not find key key for enum. Possible keys are {entry_dict.keys()}")
    if value_key is None:
        for k in entry_dict.keys():
            if k.endswith("Id"):
                value_key = k
                remaining_keys.remove(k)
                break
        if value_key is None:
            raise ValueError(f"Could not find value key for enum. Possible keys are {entry_dict.keys()}")

    return key_key, value_key, remaining_keys


In [14]:
def camel_to_snake(camel: str) -> str:
    """
    Convert a camelCase string to snake_case.
    """
    snake = re.sub(r'(?<!^)(?=[A-Z])', '_', camel).lower()
    return snake

In [ ]:
for k, v in o.items():
    entries: list[EnumEntry] = []
    cls_name = k.title()
    print(f'Processing Enum: {cls_name}')
    key_key, value_key, remaining_keys = get_keys(v[0])
    print(f"  Key: {key_key}, Value: {value_key}, Other keys: {remaining_keys}")
    for entry_dict in v:
        try:
            entry_key = entry_dict[key_key]
            entry_value = entry_dict[value_key]
        except KeyError as ke:
            print(f"Error processing entry in {cls_name}: {ke}. Available keys are: {entry_dict.keys()}")
        entry_comment = str({k: v for k, v in entry_dict.items() if k in remaining_keys}) if remaining_keys else None
        if entry_key is None:
            entry_key = 'None'
        entry = EnumEntry(key=entry_key, value=int(entry_value), comment=entry_comment)
        
        entries.append(entry)

    # for e in entries:
    #     try:
    #         print(f"  {e.to_string()}")
    #     except Exception as ex:
    #         raise ValueError(f"Failed to process entry {e}. {o[k]}") from ex

    with enum_file(fp=AUTO_ENUMS_PATH / 'wds' / f'{camel_to_snake(k)}.py', imports={'enum': 'Enum'}, overwrite=True) as f:
        write_enum_class(f, entries=entries, cls_name=cls_name, skip_methods=True)
    # print(f"Key: {key_key}, Value: {value_key}")
    # print(f"{k}: {v}")

Processing Enum: Scalar
  Key: scalarFactorDescEn, Value: scalarFactorCode, Other keys: {'scalarFactorDescFr'}


ValueError: Failed to process entry EnumEntry(key='units', value=0, comment="{'scalarFactorDescFr': 'unités'}"). [{'scalarFactorCode': 0, 'scalarFactorDescEn': 'units', 'scalarFactorDescFr': 'unités'}, {'scalarFactorCode': 1, 'scalarFactorDescEn': 'tens', 'scalarFactorDescFr': 'dizaines'}, {'scalarFactorCode': 2, 'scalarFactorDescEn': 'hundreds', 'scalarFactorDescFr': 'centaines'}, {'scalarFactorCode': 3, 'scalarFactorDescEn': 'thousands', 'scalarFactorDescFr': 'milliers'}, {'scalarFactorCode': 4, 'scalarFactorDescEn': 'tens of thousands', 'scalarFactorDescFr': 'dizaines de milliers'}, {'scalarFactorCode': 5, 'scalarFactorDescEn': 'hundreds of thousands', 'scalarFactorDescFr': 'centaines de milliers'}, {'scalarFactorCode': 6, 'scalarFactorDescEn': 'millions', 'scalarFactorDescFr': 'millions'}, {'scalarFactorCode': 7, 'scalarFactorDescEn': 'tens of millions', 'scalarFactorDescFr': 'dizaines de millions'}, {'scalarFactorCode': 8, 'scalarFactorDescEn': 'hundreds of millions', 'scalarFacto